In [13]:
import numpy as np
import pandas as pd

In [14]:
df = pd.read_csv('data/worldbank_2000_2024.csv')

In [15]:
# Rút ngắn tên các cột để dễ phân tích
df.columns = [
    'country_name', 'country_code', 'year', 'population', 'poverty_ratio',
    'pop_growth', 'life_expectancy', 'gdp_per_capita', 'gdp_growth',
    'sanitation', 'electricity', 'water_access', 'co2_emissions',
    'slum_population', 'labor_force'
]

In [16]:
len(df['sanitation'])

5425

In [17]:
df['country_name'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh',
       'Bulgaria', 'Bahrain', 'Bahamas, The', 'Bosnia and Herzegovina',
       'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Barbados',
       'Brunei Darussalam', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland',
       'Channel Islands', 'Chile', 'China', "Cote d'Ivoire", 'Cameroon',
       'Congo, Dem. Rep.', 'Congo, Rep.', 'Colombia', 'Comoros',
       'Cabo Verde', 'Costa Rica', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czechia', 'Germany', 'Djibouti', 'Dominica', 'Denmark',
       'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt, Arab Rep.',
       'Eritrea', 'Spain', 'Estonia', 'Ethiopia', 'Finland', 'Fiji',
       'France', 'Faroe Islands', 'Micronesia, Fed

In [18]:
df[df['country_name'] == 'Vanuatu']['sanitation']

5250   NaN
5251   NaN
5252   NaN
5253   NaN
5254   NaN
5255   NaN
5256   NaN
5257   NaN
5258   NaN
5259   NaN
5260   NaN
5261   NaN
5262   NaN
5263   NaN
5264   NaN
5265   NaN
5266   NaN
5267   NaN
5268   NaN
5269   NaN
5270   NaN
5271   NaN
5272   NaN
5273   NaN
5274   NaN
Name: sanitation, dtype: float64

In [19]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


feature_cols = [
    'year',
    'gdp_per_capita',
    'electricity',
    'water_access',
    'population',
    'pop_growth',
    'gdp_growth',
    'co2_emissions',
    'labor_force',
]

feature_cols = [c for c in feature_cols if c in df.columns]

def interpolate_then_median(dataframe):
    out = dataframe.copy()
    out['sanitation'] = (
        out.groupby('country_name')['sanitation']
        .apply(lambda s: s.interpolate(method='linear', limit_direction='both'))
        .reset_index(level=0, drop=True)
    )
    out['sanitation'] = out['sanitation'].fillna(
        out.groupby('country_name')['sanitation'].transform('median')
    )
    out['sanitation'] = out['sanitation'].fillna(out['sanitation'].median())
    return out


def per_country_linear_regression(dataframe):
    out = dataframe.copy()
    for country, sub in out.groupby('country_name'):
        idx = sub.index
        train = sub[sub['sanitation'].notna()]
        if len(train) < 4:
            continue
        X_train = train[feature_cols]
        y_train = train['sanitation']
        model = Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('lr', LinearRegression())
        ])
        model.fit(X_train, y_train)
        missing_mask = sub['sanitation'].isna()
        if missing_mask.any():
            X_pred = sub.loc[missing_mask, feature_cols]
            out.loc[missing_mask.index[missing_mask], 'sanitation'] = model.predict(X_pred)
    out['sanitation'] = out['sanitation'].fillna(
        out.groupby('country_name')['sanitation'].transform('median')
    )
    out['sanitation'] = out['sanitation'].fillna(out['sanitation'].median())
    return out


def global_linear_with_country(dataframe):
    out = dataframe.copy()
    train = out[out['sanitation'].notna()]
    X_train = train[feature_cols + ['country_name']]
    y_train = train['sanitation']
    pre = ColumnTransformer(
        [
            ('num', SimpleImputer(strategy='median'), feature_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), ['country_name']),
        ]
    )
    model = Pipeline([
        ('pre', pre),
        ('ridge', Ridge(alpha=1.0))
    ])
    model.fit(X_train, y_train)
    missing_mask = out['sanitation'].isna()
    if missing_mask.any():
        X_pred = out.loc[missing_mask, feature_cols + ['country_name']]
        out.loc[missing_mask, 'sanitation'] = model.predict(X_pred)
    return out


def global_random_forest(dataframe):
    out = dataframe.copy()
    train = out[out['sanitation'].notna()]
    X_train = train[feature_cols + ['country_name']]
    y_train = train['sanitation']
    pre = ColumnTransformer(
        [
            ('num', SimpleImputer(strategy='median'), feature_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), ['country_name']),
        ]
    )
    model = Pipeline([
        ('pre', pre),
        ('rf', RandomForestRegressor(
            n_estimators=300,
            random_state=42,
            n_jobs=-1,
            min_samples_leaf=2
        )),
    ])
    model.fit(X_train, y_train)
    missing_mask = out['sanitation'].isna()
    if missing_mask.any():
        X_pred = out.loc[missing_mask, feature_cols + ['country_name']]
        out.loc[missing_mask, 'sanitation'] = model.predict(X_pred)
    return out


def knn_impute(dataframe):
    out = dataframe.copy()
    knn_cols = feature_cols + ['sanitation']
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    imputed = imputer.fit_transform(out[knn_cols])
    out['sanitation'] = imputed[:, knn_cols.index('sanitation')]
    return out


def country_metrics(df_imputed):
    stats = df_imputed.groupby('country_name')['sanitation'].agg(['nunique', 'std']).reset_index()
    constant_ratio = (stats['nunique'] <= 1).mean()
    low_var_ratio = (stats['std'].fillna(0) < 0.1).mean()

    corrs = []
    for _, sub in df_imputed.groupby('country_name'):
        if sub['year'].nunique() < 3:
            continue
        x = sub['year']
        y = sub['sanitation']
        corr = x.rank().corr(y.rank())
        if pd.notna(corr):
            corrs.append(corr)
    avg_year_corr = float(np.mean(corrs)) if corrs else np.nan
    return constant_ratio, low_var_ratio, avg_year_corr



from sklearn.experimental import enable_iterative_imputer  # noqa: F401
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

mice_high_corr_cols = [
    c for c in ['water_access', 'electricity', 'gdp_per_capita', 'co2_emissions', 'sanitation']
    if c in df.columns
]

def mice_high_corr(dataframe):
    out = dataframe.copy()
    imputer = IterativeImputer(estimator=LinearRegression(), random_state=42, max_iter=20)
    imputed = imputer.fit_transform(out[mice_high_corr_cols])
    out['sanitation'] = imputed[:, mice_high_corr_cols.index('sanitation')]
    return out

methods = {
    'A_interpolate_median': interpolate_then_median,
    'B_country_lr': per_country_linear_regression,
    'C_global_lr_country': global_linear_with_country,
    'D_global_rf_country': global_random_forest,
    'E_knn_imputer': knn_impute,
    'F_mice_high_corr': mice_high_corr,
}

rng = np.random.default_rng(42)
known_idx = df[df['sanitation'].notna()].index
mask_idx = rng.choice(known_idx, size=int(len(known_idx) * 0.2), replace=False)
df_masked = df.copy()
df_masked.loc[mask_idx, 'sanitation'] = np.nan

rows = []
for name, func in methods.items():
    df_imp = func(df_masked)
    y_true = df.loc[mask_idx, 'sanitation']
    y_pred = df_imp.loc[mask_idx, 'sanitation']
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    constant_ratio, low_var_ratio, avg_year_corr = country_metrics(df_imp)
    rows.append({
        'method': name,
        'mae': mae,
        'rmse': rmse,
        'constant_ratio': constant_ratio,
        'low_var_ratio': low_var_ratio,
        'avg_year_spearman': avg_year_corr,
    })

summary = pd.DataFrame(rows).sort_values(['mae', 'constant_ratio']).reset_index(drop=True)
summary


c:\Users\DELL\.conda\envs\course\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\DELL\.conda\envs\course\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\DELL\.conda\envs\course\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['electricity' 'labor_force']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\DELL\.conda\envs\course\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['electricity' 'labor_force']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\DELL\.conda\envs\course\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed va

,method,mae,rmse,constant_ratio,low_var_ratio,avg_year_spearman
0,A_interpolate_median,0.080830,0.389010,0.396313,0.423963,0.580420
1,B_country_lr,0.389214,1.127082,0.396313,0.419355,0.522378
2,D_global_rf_country,2.236437,4.223081,0.000000,0.000000,0.472368
3,E_knn_imputer,18.793682,26.116758,0.000000,0.004608,0.240205
4,C_global_lr_country,21.044248,24.524747,0.000000,0.027650,0.469871


### ??ng b? ph??ng ph?p MICE (bi?n t??ng quan cao)
Cell n?y ??m b?o `F_mice_high_corr` lu?n c? trong `methods` tr??c khi t?o b?ng/summary.


In [ ]:
# ??m b?o ph??ng ph?p F_mice_high_corr c? trong methods
if 'F_mice_high_corr' not in methods:
    methods['F_mice_high_corr'] = mice_high_corr


### B? sung: MICE v?i c?c bi?n t??ng quan cao
Ph??ng ph?p n?y ch? d?ng c?c bi?n c? t??ng quan cao v?i `sanitation`: `water_access`, `electricity`, `gdp_per_capita`, `co2_emissions`.


In [20]:
# Simple composite ranking: lower error + lower constant_ratio + higher year trend
summary = summary.copy()
summary['rank_score'] = (
    summary['mae'].rank()
    + summary['constant_ratio'].rank()
    - summary['avg_year_spearman'].rank()
)
summary = summary.sort_values('rank_score').reset_index(drop=True)
summary


,method,mae,rmse,constant_ratio,low_var_ratio,avg_year_spearman,rank_score
0,A_interpolate_median,0.080830,0.389010,0.396313,0.423963,0.580420,0.5
1,D_global_rf_country,2.236437,4.223081,0.000000,0.000000,0.472368,2.0
2,B_country_lr,0.389214,1.127082,0.396313,0.419355,0.522378,2.5
3,E_knn_imputer,18.793682,26.116758,0.000000,0.004608,0.240205,5.0
4,C_global_lr_country,21.044248,24.524747,0.000000,0.027650,0.469871,5.0


In [21]:
# Apply best method and show sample results for inspection
best_df = global_random_forest(df)

# Show a few countries across years
sample_countries = ['Vanuatu', 'Vietnam', 'Brazil', 'Nigeria', 'India']
print(best_df[best_df['country_name'].isin(sample_countries)][
    ['country_name', 'year', 'sanitation', 'gdp_per_capita', 'electricity', 'water_access']
].sort_values(['country_name', 'year']).head(60))

# Show per-country summary stats to verify variability
summary_stats = best_df.groupby('country_name')['sanitation'].agg(['min', 'max', 'mean', 'std']).reset_index()
print(summary_stats.sort_values('std', ascending=False).head(10))


     country_name  year  sanitation  gdp_per_capita  electricity  water_access
650        Brazil  2000   35.742042     3766.548115         94.4     93.471525
651        Brazil  2001   36.167626     3176.289357         96.0     93.814344
652        Brazil  2002   36.585900     2855.940189         96.7     94.142490
653        Brazil  2003   37.002147     3090.607010         97.0     94.463526
654        Brazil  2004   37.416236     3663.823188         96.8     94.777358
655        Brazil  2005   37.828215     4827.781731         97.1     95.084136
656        Brazil  2006   38.238281     5934.144709         97.6     95.384188
657        Brazil  2007   38.646468     7409.691307         98.1     95.677627
658        Brazil  2008   39.052810     8908.331765         98.5     95.964564
659        Brazil  2009   39.457043     8678.659224         98.9     96.244799
660        Brazil  2010   39.859511    11403.282128         98.6     96.518800
661        Brazil  2011   40.261554    13396.624356 

## Bảng dữ liệu sau khi điền `sanitation` theo từng phương pháp

Mục tiêu: tạo bảng dữ liệu sau khi điền `sanitation` cho **từng phương pháp** để bạn có thể thử nghiệm trực tiếp.
Bên dưới có giải thích ngắn cho mỗi phương pháp và phần hiển thị bảng mẫu.


### Giải thích các phương pháp
- **A_interpolate_median**: Nội suy tuyến tính theo năm trong từng quốc gia, sau đó điền phần còn thiếu bằng median theo quốc gia, cuối cùng fallback median toàn cục.
- **B_country_lr**: Hồi quy tuyến tính riêng cho từng quốc gia (dựa trên các biến số) để dự đoán `sanitation`, nếu thiếu nhiều thì fallback bằng median quốc gia/toàn cục.
- **C_global_lr_country**: Hồi quy Ridge toàn cục có thêm `country_name` (one-hot) và các biến số; dùng để dự đoán `sanitation` cho toàn bộ dữ liệu.
- **D_global_rf_country**: Random Forest toàn cục có thêm `country_name` (one-hot) và các biến số; thường bắt được quan hệ phi tuyến tốt hơn.
- **E_knn_imputer**: KNN Imputer trên tập biến số + `sanitation`, dùng láng giềng gần nhất để điền thiếu.


In [22]:
df['country_name'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh',
       'Bulgaria', 'Bahrain', 'Bahamas, The', 'Bosnia and Herzegovina',
       'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Barbados',
       'Brunei Darussalam', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland',
       'Channel Islands', 'Chile', 'China', "Cote d'Ivoire", 'Cameroon',
       'Congo, Dem. Rep.', 'Congo, Rep.', 'Colombia', 'Comoros',
       'Cabo Verde', 'Costa Rica', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czechia', 'Germany', 'Djibouti', 'Dominica', 'Denmark',
       'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt, Arab Rep.',
       'Eritrea', 'Spain', 'Estonia', 'Ethiopia', 'Finland', 'Fiji',
       'France', 'Faroe Islands', 'Micronesia, Fed

In [23]:
# T?o b?ng d? li?u sau khi ?i?n sanitation theo t?ng ph??ng ph?p

# M?i b?ng s? hi?n th? m?t s? c?t ch?nh ?? ti?n so s?nh
show_cols = [
    'country_name', 'country_code', 'year', 'sanitation',
    'electricity', 'water_access', 'gdp_per_capita'
]
show_cols = [c for c in show_cols if c in df.columns]

imputed_tables = {}
for name, func in methods.items():
    df_imp = func(df)
    imputed_tables[name] = df_imp

# Hi?n th? b?ng m?u (head) cho t?ng ph??ng ph?p
for name, df_imp in imputed_tables.items():
    print(f"===== {name} =====")
    new_df = df_imp[show_cols]
    display(new_df[new_df['country_name'] == 'Japan']['sanitation'])


c:\Users\DELL\.conda\envs\course\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['electricity' 'labor_force']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\DELL\.conda\envs\course\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['electricity' 'labor_force']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\DELL\.conda\envs\course\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['co2_emissions' 'labor_force']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\DELL\.conda\envs\course\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['co2_emissions' 'labor_force']. At least one non-miss

===== A_interpolate_median =====


2450    96.404380
2451    96.552311
2452    96.694026
2453    96.831958
2454    96.966109
2455    97.096478
2456    97.223066
2457    97.397812
2458    97.567347
2459    97.731657
2460    97.890727
2461    98.044545
2462    98.193096
2463    98.336366
2464    98.474341
2465    98.607007
2466    98.734351
2467    98.856358
2468    98.921992
2469    98.979321
2470    99.034890
2471    99.088703
2472    99.140756
2473    99.140756
2474    99.140756
Name: sanitation, dtype: float64

===== B_country_lr =====


2450    96.404380
2451    96.552311
2452    96.694026
2453    96.831958
2454    96.966109
2455    97.096478
2456    97.223066
2457    97.397812
2458    97.567347
2459    97.731657
2460    97.890727
2461    98.044545
2462    98.193096
2463    98.336366
2464    98.474341
2465    98.607007
2466    98.734351
2467    98.856358
2468    98.921992
2469    98.979321
2470    99.034890
2471    99.088703
2472    99.140756
2473    98.588168
2474    98.743586
Name: sanitation, dtype: float64

...

2450    96.404380
2451    96.552311
2452    96.694026
2453    96.831958
2454    96.966109
2455    97.096478
2456    97.223066
2457    97.397812
2458    97.567347
2459    97.731657
2460    97.890727
2461    98.044545
2462    98.193096
2463    98.336366
2464    98.474341
2465    98.607007
2466    98.734351
2467    98.856358
2468    98.921992
2469    98.979321
2470    99.034890
2471    99.088703
2472    99.140756
2473    64.796812
2474    63.869304
Name: sanitation, dtype: float64

===== D_global_rf_country =====


2450    96.404380
2451    96.552311
2452    96.694026
2453    96.831958
2454    96.966109
2455    97.096478
2456    97.223066
2457    97.397812
2458    97.567347
2459    97.731657
2460    97.890727
2461    98.044545
2462    98.193096
2463    98.336366
2464    98.474341
2465    98.607007
2466    98.734351
2467    98.856358
2468    98.921992
2469    98.979321
2470    99.034890
2471    99.088703
2472    99.140756
2473    91.243558
2474    72.386132
Name: sanitation, dtype: float64

===== E_knn_imputer =====


2450    96.404380
2451    96.552311
2452    96.694026
2453    96.831958
2454    96.966109
2455    97.096478
2456    97.223066
2457    97.397812
2458    97.567347
2459    97.731657
2460    97.890727
2461    98.044545
2462    98.193096
2463    98.336366
2464    98.474341
2465    98.607007
2466    98.734351
2467    98.856358
2468    98.921992
2469    98.979321
2470    99.034890
2471    99.088703
2472    99.140756
2473    55.598735
2474    58.230364
Name: sanitation, dtype: float64

### Gợi ý sử dụng
Bạn có thể đổi `head(20)` thành `head(100)` hoặc bỏ `head(...)` để xem toàn bộ bảng (nếu máy chịu tải được). Nếu cần lưu ra CSV để test riêng, cho mình biết mình sẽ thêm cell export.


### Xuất CSV cho từng phương pháp
Cell dưới đây sẽ lưu file CSV của từng phương pháp vào thư mục `data/processed/imputed_methods/`.


In [24]:
import os

output_dir = "data/processed/imputed_methods"
os.makedirs(output_dir, exist_ok=True)

for name, df_imp in imputed_tables.items():
    out_path = os.path.join(output_dir, f"sanitation_imputed_{name}.csv")
    df_imp.to_csv(out_path, index=False)
    print(f"Đã lưu: {out_path}")


Đã lưu: data/processed/imputed_methods\sanitation_imputed_A_interpolate_median.csv
Đã lưu: data/processed/imputed_methods\sanitation_imputed_B_country_lr.csv
Đã lưu: data/processed/imputed_methods\sanitation_imputed_C_global_lr_country.csv
Đã lưu: data/processed/imputed_methods\sanitation_imputed_D_global_rf_country.csv
Đã lưu: data/processed/imputed_methods\sanitation_imputed_E_knn_imputer.csv


In [25]:
df_interpolate = pd.read_csv('data/processed/imputed_methods\sanitation_imputed_A_interpolate_median.csv')
df_country_lr = pd.read_csv('data/processed/imputed_methods\sanitation_imputed_B_country_lr.csv')
df_global_lr = pd.read_csv('data/processed/imputed_methods\sanitation_imputed_C_global_lr_country.csv')
df_global_rf = pd.read_csv('data/processed/imputed_methods\sanitation_imputed_D_global_rf_country.csv')
df_knn_imputer = pd.read_csv('data/processed/imputed_methods\sanitation_imputed_E_knn_imputer.csv')

In [26]:
df['country_name'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh',
       'Bulgaria', 'Bahrain', 'Bahamas, The', 'Bosnia and Herzegovina',
       'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Barbados',
       'Brunei Darussalam', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland',
       'Channel Islands', 'Chile', 'China', "Cote d'Ivoire", 'Cameroon',
       'Congo, Dem. Rep.', 'Congo, Rep.', 'Colombia', 'Comoros',
       'Cabo Verde', 'Costa Rica', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czechia', 'Germany', 'Djibouti', 'Dominica', 'Denmark',
       'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt, Arab Rep.',
       'Eritrea', 'Spain', 'Estonia', 'Ethiopia', 'Finland', 'Fiji',
       'France', 'Faroe Islands', 'Micronesia, Fed

In [ ]:
print(df_interpolate[df_interpolate['country_name'] == 'Argentina']['sanitation'])
print()
print(df_country_lr[df_country_lr['country_name'] == 'Argentina']['sanitation'])
print()
print(df_global_lr[df_global_lr['country_name'] == 'Argentina']['sanitation'])
print()
print(df_global_rf[df_global_rf['country_name'] == 'Grenada']['sanitation'])
print()
print(df_knn_imputer[df_knn_imputer['country_name'] == 'Grenada']['sanitation'])

1925    51.957266
1926    51.957266
1927    51.957266
1928    51.957266
1929    51.957266
1930    51.957266
1931    51.957266
1932    51.957266
1933    51.957266
1934    51.957266
1935    51.957266
1936    51.957266
1937    51.957266
1938    51.957266
1939    51.957266
1940    51.957266
1941    51.957266
1942    51.957266
1943    51.957266
1944    51.957266
1945    51.957266
1946    51.957266
1947    51.957266
1948    51.957266
1949    51.957266
Name: sanitation, dtype: float64

1925    51.545345
1926    51.545345
1927    51.545345
1928    51.545345
1929    51.545345
1930    51.545345
1931    51.545345
1932    51.545345
1933    51.545345
1934    51.545345
1935    51.545345
1936    51.545345
1937    51.545345
1938    51.545345
1939    51.545345
1940    51.545345
1941    51.545345
1942    51.545345
1943    51.545345
1944    51.545345
1945    51.545345
1946    51.545345
1947    51.545345
1948    51.545345
1949    51.545345
Name: sanitation, dtype: float64

1925    45.649426
1926    45.635

In [42]:
(df_interpolate['sanitation'] < 0).sum()


np.int64(0)

In [45]:
(df_global_lr['sanitation'] < 0).sum()


np.int64(0)

In [46]:
(df_global_rf['sanitation'] < 0).sum()

np.int64(0)